In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np

In [0]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/comp/train.csv")

In [0]:
df = df.drop(["itemid"],axis=1)

In [5]:
df = df.loc[(df['Category'] <= 16)]
df.head()

,title,Category,image_path
0,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1,etude house precious mineral any cushion pearl...,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg


In [0]:
import string
import re

word_dict = dict()

def add_word_dict(y):
  for w in y.split():
    if not w in word_dict:
      word_dict[w] = 1
    else:
      word_dict[w] += 1

def remove_row(y):
  y = change_punc(y)
  temp = []
  for x in y.split():
    x = remove_digit(x)
    x = reduce_lengthening(x)
    temp.append(x)
  clean = ' '.join(temp)
  clean = re.sub(' {2,}', ' ', clean)
  return(clean.strip())

def remove_digit(w):
  result = ''.join([i for i in w if not i.isdigit()])
  return result
#   table = result.maketrans({key: None for key in string.punctuation})
#   result = result.translate(table)
def change_punc(result):
  clean = re.sub(r"[,.;:@#?!%&$+-]+\ *", " ", result)
  return clean
def reduce_lengthening(text):
  pattern = re.compile(r"(.)\1{2,}")
  return pattern.sub(r"\1\1", text)

In [0]:
#data cleaning for each row
df["title"] = df["title"].apply(remove_row)

In [8]:
all_words = set()

def word_set(y):
  for word in y.split():
    if word not in all_words:
      all_words.add(word)

df["title"].apply(word_set)
print(len(all_words))

20151


In [9]:
length_list = []
for l in df.title:
  length_list.append(len(l.split(" ")))
input_max_len = np.max(length_list)
print(input_max_len)

19


In [10]:
input_words = sorted(list(all_words))
num_encoder_tokens = len(input_words) + 1
input_token_index = dict([(word,i+1) for i, word in enumerate(input_words)])
input_token_index["<blank>"] = 0
print(input_token_index)

{'a': 1, 'aa': 2, 'aacs': 3, 'aadc': 4, 'aag': 5, 'aamo': 6, 'aaw': 7, 'ab': 8, 'ababa': 9, 'abadi': 10, 'abal': 11, 'abalabal': 12, 'abalone': 13, 'abang': 14, 'abc': 15, 'abe': 16, 'abh': 17, 'ability': 18, 'abishopid': 19, 'abisin': 20, 'abiss': 21, 'abjad': 22, 'able': 23, 'abm': 24, 'abody': 25, 'about': 26, 'above': 27, 'abrasion': 28, 'abricot': 29, 'abs': 30, 'absolu': 31, 'absolue': 32, 'absolut': 33, 'absolute': 34, 'absolutely': 35, 'absorb': 36, 'absorbent': 37, 'absorber': 38, 'absorbing': 39, 'absorder': 40, 'absorption': 41, 'absreinvent': 42, 'abu': 43, 'abudhabi': 44, 'abyss': 45, 'ac': 46, 'acaci': 47, 'acai': 48, 'acak': 49, 'acane': 50, 'acanthe': 51, 'acb': 52, 'acblk': 53, 'acc': 54, 'accesories': 55, 'accessory': 56, 'accusing': 57, 'acd': 58, 'ace': 59, 'acelora': 60, 'acena': 61, 'acerola': 62, 'achne': 63, 'achromatus': 64, 'acial': 65, 'acid': 66, 'acific': 67, 'acirnigbiba': 68, 'acitve': 69, 'acl': 70, 'aclarador': 71, 'acliri': 72, 'aclosy': 73, 'acmedica'

In [0]:
df = df.sample(frac=1.)

In [0]:
input_data = np.zeros(
(len(df.title),input_max_len),dtype="float32")
target_data = df["Category"].values
image_data = df["image_path"].values

In [0]:
for i,input_text in enumerate(df.title):
  for t,word in enumerate(input_text.split()):
    input_data[i,t] = input_token_index[word]

In [14]:
embedding_size = 100

from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


In [0]:
def generate_embedding_matrix(w2v_path1,w2v_path2, dec_map, lang_dim = 100):
  embeddings_index1 = {}
  embeddings_index2 = {}
  f1 = open (w2v_path1,"r",encoding = "utf-8")
  f2 = open (w2v_path2,"r",encoding = "utf-8")
  for line in f1:
    values = line.split()
    word= values[0]
    coefs = np.asarray(values[1:],dtype="float32")
    embeddings_index1[word] = coefs
  f1.close()
  for line in f2:
    values = line.split()
    word= values[0]
    coefs = np.asarray(values[1:],dtype="float32")
    embeddings_index2[word] = coefs
  f2.close()
  #prepare embedding_matrix
  embedding_matrix = np.random.rand(len(dec_map),lang_dim)
  for idx, wd in dec_map.items():
    if wd in embeddings_index1.keys():
      embedding_matrix[idx] = embeddings_index1[wd]
    elif (wd in embeddings_index2.keys()):
      embedding_matrix[idx] = embeddings_index2[wd]
    else:
      continue
  return embedding_matrix

In [16]:
idx2word = {}
for word, index in input_token_index.items():
  idx2word[index] = word
print(len(idx2word))
embedding_matrix = generate_embedding_matrix("/content/gdrive/My Drive/comp/glove.6B.100d.txt","/content/gdrive/My Drive/comp/custom_glove_100d.txt",idx2word)

20152


In [0]:
from sklearn.model_selection import train_test_split

input_text_train, input_text_test, input_image_train, input_image_text, label_train, label_text = train_test_split(input_data, image_data, target_data,test_size=0.05, random_state=88)

In [0]:
GENERATOR_BATCH_SIZE = 64

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import imageio
import imgaug as ia

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten,GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate 
from keras.layers import GRU, Dense, BatchNormalization, Input, Conv1D, SpatialDropout1D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model

In [0]:
def product_generator(input_data,input_image, input_labels):
  iteration = 0

  while True:
    
    if iteration >= (len(input_data) / GENERATOR_BATCH_SIZE):
      iteration = 0
    
    start = iteration * GENERATOR_BATCH_SIZE
    end = (iteration + 1) * GENERATOR_BATCH_SIZE - 1
    
    batch_text_data = input_data[start:end,]
    raw_batch_image_data = input_image[start:end,]
    batch_labels = input_labels[start:end,]
    batch_image_data = []
    for image_path in raw_batch_image_data:
      try:
        img = image.load_img("/content/gdrive/My Drive/"+image_path, target_size=(229, 229))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        batch_image_data.append(x[0])
      except Exception as e:
        print(e)
    batch_image_data = np.asarray(batch_image_data)
    yield ([ batch_text_data, batch_image_data], batch_labels)
    iteration += 1

In [0]:
def build_text_model():
  dropout_rate = 0.3
  units = 100
  text_input = Input(shape=(input_max_len,))
  x = Embedding(num_encoder_tokens,embedding_size,weights=[embedding_matrix])(text_input)
  x1 =  SpatialDropout1D(dropout_rate)(x)

  x_gru = Bidirectional(GRU(units, return_sequences = True))(x1)
  x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_gru)
  avg_pool1_gru = GlobalAveragePooling1D()(x1)
  max_pool1_gru = GlobalMaxPooling1D()(x1)

  x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_gru)
  avg_pool3_gru = GlobalAveragePooling1D()(x3)
  max_pool3_gru = GlobalMaxPooling1D()(x3)

  x_lstm = Bidirectional(LSTM(units, return_sequences = True))(x1)
  x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_lstm)
  avg_pool1_lstm = GlobalAveragePooling1D()(x1)
  max_pool1_lstm = GlobalMaxPooling1D()(x1)

  x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
  avg_pool3_lstm = GlobalAveragePooling1D()(x3)
  max_pool3_lstm = GlobalMaxPooling1D()(x3)

  x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                      avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
  x = BatchNormalization()(x)
  x = Dropout(0.2)(Dense(128,activation='relu') (x))
  
  return x, text_input

In [0]:
def build_image_model():
  base_model = InceptionV3(weights='imagenet', include_top=False)

   # Freeze Inception's weights - we don't want to train these
  for layer in base_model.layers:
     layer.trainable = False

  # add a fully connected layer after Inception - we do want to train these
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(2048, activation='relu')(x)

  return x, base_model.input

In [0]:
text_x, text_input = build_text_model()
image_x, image_input = build_image_model()

joint_x = concatenate([ text_x, image_x ])
joint_x = Dense(512, activation='relu')(joint_x)
predictions = Dense(58, activation='sigmoid')(joint_x)

full_model = Model(inputs=[ text_input, image_input ], outputs=[ predictions ])

full_model.compile(optimizer=Adam(lr = 0.001, decay = 1e-5),loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [0]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',np.unique(target_data),target_data)

In [0]:
file_path = "model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_acc", verbose = 1,
                              save_best_only = True, mode = "max")
early_stop = EarlyStopping(monitor = "val_acc", mode = "max", patience = 4)

full_model.fit_generator(product_generator(input_text_train, input_image_train,label_train),
                         validation_data=product_generator(input_text_test, input_image_test,label_test),
                         validation_steps=len(input_text_test)/GENERATOR_BATCH_SIZE,
                         steps_per_epoch=len(input_text_train)/GENERATOR_BATCH_SIZE,
                         class_weight=class_weight,epochs=50)